# import modules

In [2]:
#1
!pip install -q -U json-lines

import json
import os
import pandas as pd
import tarfile
import json_lines
from tqdm.notebook import tqdm
from pprint import pprint
import glob
import matplotlib.pyplot as plt
import numpy as np

# read party, tweets, mentions and hashtags

In [3]:
#2
#funktioniert: get Party name + Tweets
files123 = glob.glob('./data/*.jl')

alles = []
error = []

AfD = 0
Grünen = 0
CDU = 0
CSU = 0
Linke = 0
FDP = 0
SPD = 0
maxWert = 50000

i = 0

for file in tqdm(files123): # responsible for visual output
    
    with json_lines.open(file) as f:
        for item in f:
            try:
                if item['response']['meta']['result_count'] == 0: 
                    continue
                elif item['account_data']['Partei'] == "Fraktionslos":
                    continue
                if item['http_status'] == 200:
                    
                    #get party name
                    partei = item["account_data"]["Partei"]
                    
                    #get all tweets per person
                    tweets_array = item["response"]["data"]
                    
                    for tweet in tweets_array: #range(0,len(tweets_array)-1):
                        hashtagArray = ""
                        mentionArray = ""
                        
                       # print(type(tweet))
                        if("referenced_tweets" in tweet): # tweet is a dictionary
                            tweet_type = tweet["referenced_tweets"]
                            if(tweet_type[0]["type"] == "retweeted"):
                                continue
                        
                        if("entities" in tweet): 
                            entities = tweet["entities"]
                            if("mentions" in entities):
                                mentions = entities["mentions"]
                                for ment in mentions:
                                        mention = ment["username"]
                                        mentionArray = mentionArray + " " + mention
                            if("hashtags" in entities):
                                hashtags = entities["hashtags"]
                                for tag in hashtags:
                                        hashtag = tag["tag"]
                                        hashtagArray = hashtagArray + " " + hashtag
                
                            if(not (hashtagArray == "" and mentionArray == "")):
                        
                                #if (partei == "AfD" and AfD < maxWert):
                                    #AfD += 1
                                #elif (partei == "Bündnis 90/Die Grünen" and Grünen < maxWert):
                                    #Grünen += 1
                                #elif (partei == "CDU" and CDU < maxWert):
                                    #CDU += 1
                                #elif (partei == "CSU" and CSU < maxWert):
                                    #CSU += 1
                                #elif (partei == "Die Linke" and Linke < maxWert):
                                    #Linke += 1
                                #elif (partei == "FDP" and FDP < maxWert):
                                    #FDP += 1
                                #elif (partei == "SPD" and SPD < maxWert):
                                    #SPD += 1
                                #else:
                                    #continue
                                    
                                alles.append({"Partei": partei, "hashtags_mentions": hashtagArray + " " + mentionArray})
                
                else:
                    error.append(item)
                    print("yikes")
                    
            except Exception as e:
                raise e
                           
print("Number of error-requests:", len(error))


Number of error-requests: 0


# Data Preprocessing

In [3]:
#imports
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split

#import ML LIB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

#import Evaluation Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [4]:
alles_df = pd.DataFrame(alles)
alles_df

,Partei,hashtags_mentions
0,Bündnis 90/Die Grünen,green_clarity99
1,Bündnis 90/Die Grünen,GreenDeal handelsblatt peteraltmaier GrosseE...
2,Bündnis 90/Die Grünen,peter_simone RenateKuenast
3,Bündnis 90/Die Grünen,RenateKuenast
4,Bündnis 90/Die Grünen,BernhardPoetter
...,...,...
1336715,CDU,BMBF_Bund
1336716,CDU,BMBF_Bund
1336717,CDU,fz_juelich RWTH BMBF_Bund
1336718,CDU,EKD diakonie


In [5]:
alles_df["Partei"].value_counts()

Bündnis 90/Die Grünen    355805
Die Linke                245403
SPD                      234855
CDU                      228792
FDP                      137002
AfD                       79309
CSU                       55554
Name: Partei, dtype: int64

In [6]:
len(alles_df)

1336720

In [7]:
#split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(alles_df["hashtags_mentions"], alles_df.Partei, test_size = 0.3, random_state=111)

In [8]:
#X_train.value_counts()
X_train

1329499                                         Wahlrecht_de
1244926                                CDU Berlin Weihnacht 
840251                                              talwazir
485158                                 Greta Göttingen Bahn 
1145065     HambacherWald RLP Klimaschutz Energiewende No...
                                 ...                        
942761                                             nouripour
102486                    Klimaschutz Solar Afrika Sonne Öl 
135892                                               Wissing
534484                                         BotschafterDK
654188                                           USWahl2016 
Name: hashtags_mentions, Length: 935704, dtype: object

In [9]:
y_train

1329499    Bündnis 90/Die Grünen
1244926                      CDU
840251     Bündnis 90/Die Grünen
485158                       AfD
1145065    Bündnis 90/Die Grünen
                   ...          
942761                       CDU
102486     Bündnis 90/Die Grünen
135892     Bündnis 90/Die Grünen
534484                       CDU
654188                       SPD
Name: Partei, Length: 935704, dtype: object

In [10]:
stemmer = nltk.stem.SnowballStemmer("german")
nltk.download("stopwords")
stop_words=set(nltk.corpus.stopwords.words("german"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Runjie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#write our own tokenizer function
def tokenize(text):
    tokens = [word for word in nltk.word_tokenize(text)]
    tokens = map(str.lower, tokens)
    tokens = [word.strip(" ") for word in tokens]
    tokens = [item for item in tokens if (item not in stop_words)]
    return tokens

# Support Vector

## Hashtags + Mentions

In [ ]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=50000)),
                     ("clf", LinearSVC(C=1.0, penalty="l1", max_iter=6000, dual=False))])

In [ ]:
model = pipeline.fit(X_train, y_train)

In [ ]:
vectorizer = model.named_steps["vect"]
chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [ ]:
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices = True)]
feature_names = np.asarray(feature_names)

In [ ]:
#target_names = ["Bündnis 90/Die Grünen", "SPD", "CDU", "Die Linke", "FDP", "AfD", "CSU"]
target_names = clf.classes_
print("top 30 keywords per class:")
for i, label in enumerate(target_names):
    top30 = np.argsort(clf.coef_[i])[-30:]
    print("%s: %s" % (label, " ".join(feature_names[top30])))
    print("\n")

In [ ]:
prediction = model.predict(X_test)

In [ ]:
print("******SVM classification report (hashtags + mentions, 50k per party)******")
print("accuracy:", accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

In [ ]:
print("************************SVM confusion matrix (hashtags + mentions, 50k per party)************************")
cm = pd.DataFrame(
    confusion_matrix(y_test, prediction, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

# Decision Tree

In [ ]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", DecisionTreeClassifier(random_state = 111))])

In [ ]:
model = pipeline.fit(X_train, y_train)

In [ ]:
vectorizer = model.named_steps["vect"]
chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [ ]:
print(clf.feature_importances_[])

In [ ]:
target_names = clf.classes_
print("leaves:", clf.get_n_leaves())
print("depth:", clf.get_depth())

In [ ]:
prediction = model.predict(X_test)

In [ ]:
print("******DecisionTree classification report (hashtags + mentions, 50k per party)******")
print("accuracy:", accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

In [ ]:
print("************************DecisionTree confusion matrix (hashtags + mentions, 50k per party)************************")
cm = pd.DataFrame(
    confusion_matrix(y_test, prediction, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

# RandomForest

In [ ]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     #("chi", SelectKBest(chi2, k=5000)),
                     ("clf", RandomForestClassifier(random_state = 111))])

In [ ]:
model = pipeline.fit(X_train, y_train)

In [ ]:
vectorizer = model.named_steps["vect"]
chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [ ]:
prediction = model.predict(X_test)

In [ ]:
target_names = clf.classes_

In [ ]:
print("******RandomForest classification report (hashtags + mentions, 50k per party)******")
print("accuracy:", accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

In [ ]:
print("************************RandomForest confusion matrix (hashtags + mentions, 50k per party)************************")
cm = pd.DataFrame(
    confusion_matrix(y_test, prediction, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

# GradientBoostingClassifier

In [58]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     #("chi", SelectKBest(chi2, k=10000)),
                     ("clf", GradientBoostingClassifier(random_state = 111))])

In [59]:
model = pipeline.fit(X_train, y_train)

In [60]:
vectorizer = model.named_steps["vect"]
#chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [61]:
prediction = model.predict(X_test)

In [62]:
target_names = clf.classes_

In [63]:
print("******GradientBoosting classification report (hashtags + mentions, all data)******")
print("accuracy:", accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

******GradientBoosting classification report (hashtags + mentions, all data)******
accuracy: 0.42450425918167856
                       precision    recall  f1-score   support

                  AfD     0.8040    0.4593    0.5846     23977
Bündnis 90/Die Grünen     0.3228    0.9236    0.4784    106610
                  CDU     0.7162    0.2144    0.3300     68602
                  CSU     0.7705    0.1533    0.2557     16828
            Die Linke     0.7914    0.2187    0.3427     73693
                  FDP     0.7755    0.2645    0.3945     40827
                  SPD     0.6859    0.2348    0.3499     70479

             accuracy                         0.4245    401016
            macro avg     0.6952    0.3527    0.3908    401016
         weighted avg     0.6337    0.4245    0.3940    401016



In [64]:
print("************************GradientBoosing confusion matrix (hashtags + mentions, all data)************************")
cm = pd.DataFrame(
    confusion_matrix(y_test, prediction, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

************************GradientBoosing confusion matrix (hashtags + mentions, all data)************************


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,11012,11359,517,11,337,207,534
true:Bündnis 90/Die Grünen,864,98467,1823,178,1840,1009,2429
true:CDU,475,49718,14707,221,718,425,2338
true:CSU,58,12361,722,2579,159,569,380
true:Die Linke,502,54918,758,60,16117,318,1020
true:FDP,343,27525,662,137,481,10800,879
true:SPD,443,50665,1347,161,713,599,16551


# KNeighborsClassifier

In [47]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     #("chi", SelectKBest(chi2, k=10000)),
                     ("clf", KNeighborsClassifier())])

In [48]:
model = pipeline.fit(X_train, y_train)

In [50]:
vectorizer = model.named_steps["vect"]
#chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [51]:
prediction = model.predict(X_test)

In [52]:
print("***************KNeighbors classification report (hashtags + mentions, all data)***************")
print("accuracy:", accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

***************KNeighbors classification report (hashtags + mentions, all data)***************
accuracy: 0.5313478763939593
                       precision    recall  f1-score   support

                  AfD     0.4904    0.4601    0.4747     23977
Bündnis 90/Die Grünen     0.4884    0.6899    0.5719    106610
                  CDU     0.5040    0.5199    0.5118     68602
                  CSU     0.4577    0.3227    0.3785     16828
            Die Linke     0.6294    0.5206    0.5698     73693
                  FDP     0.5839    0.3845    0.4637     40827
                  SPD     0.5802    0.4730    0.5211     70479

             accuracy                         0.5313    401016
            macro avg     0.5334    0.4815    0.4988    401016
         weighted avg     0.5417    0.5313    0.5274    401016



In [53]:
print("***KNeighbors confusion matrix (hashtags + mentions, all data)***")
cm = pd.DataFrame(
    confusion_matrix(y_test, prediction, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

***KNeighbors confusion matrix (hashtags + mentions, all data)***


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,11032,5601,2481,255,2255,816,1537
true:Bündnis 90/Die Grünen,2720,73549,10041,1565,8034,2984,7717
true:CDU,2382,17278,35665,1380,3728,2217,5952
true:CSU,398,4949,2885,5431,887,1000,1278
true:Die Linke,2311,19248,6592,966,38366,1764,4446
true:FDP,1667,11585,4950,985,2746,15698,3196
true:SPD,1988,18367,8152,1285,4945,2404,33338


# Multilayer Perceptron Classifier

In [23]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", MLPClassifier(hidden_layer_sizes=(7,7), max_iter = 6000, random_state = 111))])

In [27]:
model = pipeline.fit(X_train, y_train)

In [28]:
vectorizer = model.named_steps["vect"]
chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [29]:
prediction = model.predict(X_test)

In [30]:
print("number of layers:", clf.n_layers_)

number of layers: 4


In [32]:
target_names = clf.classes_

In [35]:
print("******Multilayer Perceptron classification report (hashtags + mentions, all data)******")
print("accuracy:", accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

******Multilayer Perceptron classification report (hashtags + mentions, all data)******
accuracy: 0.5613815907594709
                       precision    recall  f1-score   support

                  AfD     0.7269    0.6310    0.6756     23977
Bündnis 90/Die Grünen     0.4431    0.7823    0.5658    106610
                  CDU     0.5925    0.4786    0.5295     68602
                  CSU     0.7114    0.3250    0.4462     16828
            Die Linke     0.6852    0.5181    0.5901     73693
                  FDP     0.7023    0.4417    0.5423     40827
                  SPD     0.6755    0.4551    0.5438     70479

             accuracy                         0.5614    401016
            macro avg     0.6481    0.5188    0.5562    401016
         weighted avg     0.6086    0.5614    0.5593    401016



In [36]:
print("***Multilayer Perceptron (two hidden layer, each has 7 neurons) confusion matrix (hashtags + mentions, all data)***")
cm = pd.DataFrame(
    confusion_matrix(y_test, prediction, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

***Multilayer Perceptron (two hidden layer, each has 7 neurons) confusion matrix (hashtags + mentions, all data)***


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,15130,4591,1542,20,1257,674,763
true:Bündnis 90/Die Grünen,1365,83398,6640,663,7365,2064,5115
true:CDU,1145,25227,32836,627,3119,1487,4161
true:CSU,170,6034,2682,5469,523,1140,810
true:Die Linke,1162,27746,3305,154,38181,727,2418
true:FDP,899,14702,2867,468,1716,18035,2140
true:SPD,942,26510,5552,287,3561,1553,32074
